In [1]:
import preprocess as prep
import pandas as pd

in_path = './../../../'
dataset = pd.read_csv(
    in_path+'splunk_data_180918_telenor_processed.txt',  
    encoding="ISO-8859-1", 
    dtype={
        "user_id": int, 
        "visit_id": int, 
        "sequence": int, 
        "start_time":object, 
        "event_duration":float,
        "url":str, 
        "action":str, 
        "country":str,
        "user_client":str,
        "user_client_family":str,
        "user_experience":str,
        "user_os":str,
        "apdex_user_experience":str,
        "bounce_rate":float,
        "session_duration":float
    }
)

t = dataset.tail(4000)

t.columns = t.columns.str.replace('min_bedrift_event.','')
t = t[~t.action.isnull()]
t = t.dropna(axis='rows', how='any',subset=['url', 'action'])
t = t.reset_index()

[nltk_data] Downloading package punkt to /home/aksel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/aksel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
t = prep.clean_actions(t)

Pandas Apply: 100%|██████████| 3194/3194 [00:00<00:00, 14596.64it/s]


In [13]:
#pd.set_option('display.max_rows', len(t))


In [3]:
#t.action.value_counts().tail(100)
other = t[t['action_cleaned']=='click_on_other'].action.unique()
print(len(other))
other


292


array(['click on "REGION FINNMARK"',
       'click on "OSLO KOMMUNE BRANN- OG REDNINGSETATEN"',
       'click on "Installert i"',
       'click on "Sist brukt16.09.2018Installert iOption NV GTM378E"',
       'click on "7080000243389 - MÃ\x83Â\x98LLH. Ã\x83Â\x98KERN"',
       'click on "60 Abonnement"',
       'scroll on "926 45 344FJELD, ROBERT133006/Fjeld, Robert04.06.2018AVD MARKETING-PU908 93 596KITTELSEN, GUNNAR1955..."',
       'click on "77 63 23 94"', 'click on "post@atprofilering.no"',
       'click on "DAVID ANAND GOKSEM  951 31 231"', 'click on "Lokasjon"',
       'click on "Created with Highcharts 4.2.7Kostnader siden mÃ\x83Â¥nedsskifteFaste:263,-Faste:263,-Faste:263,-PÃ\x83Â¥lÃ\x83Â¸pt:63,..."',
       'click on "77 65 07 27"', 'click on "LIFE IMPROVEMENT AS"',
       'click on "GÃ\x83Â¥ tilbake til forsiden"',
       'click on "ANDERSEN, ROY DRAGE"', 'click on "aug. - sep."',
       'click on "3 Abonnement"', 'click on "77 84 16 87"',
       'click on "Sist brukt17.09.2018

In [4]:
import re

def quoted_text(inputString):
    return re.findall(r'"([^"]*)"', inputString)[0]

def upper(df):
    uppers = []
    for s in df:
        quoted = quoted_text(s)
        if quoted.isupper():
            uppers.append(quoted)
    return uppers

uppers = upper(other)
print('num upper case:',len(uppers))
for u in uppers:
    print(u)

num upper case: 109
REGION FINNMARK
OSLO KOMMUNE BRANN- OG REDNINGSETATEN
7080000243389 - MÃÂLLH. ÃÂKERN
DAVID ANAND GOKSEM  951 31 231
LIFE IMPROVEMENT AS
ANDERSEN, ROY DRAGE
ALTECH AS
HAUGEMYRA
INTERIÃÂRRIKET ALTA AS
KNUT EGIL OHNSTAD
NATASJA BOURS VAN DEN BRINK
HAMRE, PAUL
STEINAR HESTHAG
HJELTNES, GURI
1503 AREALSEKSJONEN
GJELDOKK, HÃÂVARD
BICO BYGG OG INNREDNING AS
1504 TILSYNSSEKSJONEN
PALETTE SOFTWARE AS
PETTER LORANGE HANSEN
HANSEN, NILS PETTER LORANGE
STANDAL, JOAKIM STRANDBERG
HOVDA, GEIR TREVERKET
OMNES, PÃÂL HELGE
SHM ENABLING TECHNOLOGIES AS
NORDEA BANK AB (PUBL), FILIAL I NORGE
LANDKREDITT BANK AS
HATLAND, MIKKEL KRISTOFFER
KLINIKK FOR FOTTERAPI, AUTORISERT FOTTERAPEUT EVA
AUTOLIFT AS
VIKT CONSULTING AS
MOLDEN, STIAN
SYVERUD, NINA MARLEN
JOHANSEN, KRISTINE IDA
LAUGERUD, ARNE
JOHANSEN, SINA MARLÃÂN
AAKVIK, RUNAR
SCHINDLER AS
GRIMSÃÂEN
MAY BRITT HOLTER
GRIMSÃÂEN, CECILIE
HÃÂRSVEIS'N AS
NORDLAND FYLKESKOMMUNE
OFFSHORE LAGER
600071000 ÃÂKONOMI
HARR, ÃÂGE

In [10]:
import spacy
import string
import en_core_web_sm

nlp = en_core_web_sm.load() 

def tag(model,action):
    doc = model(action)
    return [(X.text, X.label_) for X in doc.ents]

for u in uppers:
    print(u,tag(nlp,u))

REGION FINNMARK []
OSLO KOMMUNE BRANN- OG REDNINGSETATEN []
7080000243389 - MÃÂLLH. ÃÂKERN [('7080000243389 - MÃ\x83Â\x98LLH', 'CARDINAL')]
DAVID ANAND GOKSEM  951 31 231 [('DAVID ANAND', 'PERSON'), ('951', 'CARDINAL'), ('231', 'CARDINAL')]
LIFE IMPROVEMENT AS []
ANDERSEN, ROY DRAGE []
ALTECH AS [('ALTECH', 'ORG')]
HAUGEMYRA []
INTERIÃÂRRIKET ALTA AS []
KNUT EGIL OHNSTAD []
NATASJA BOURS VAN DEN BRINK [('NATASJA', 'ORG')]
HAMRE, PAUL [('HAMRE', 'ORG'), ('PAUL', 'ORG')]
STEINAR HESTHAG []
HJELTNES, GURI [('HJELTNES', 'GPE')]
1503 AREALSEKSJONEN [('1503', 'DATE'), ('AREALSEKSJONEN', 'ORG')]
GJELDOKK, HÃÂVARD [('HÃ\x83Â', 'ORG'), ('VARD', 'PERSON')]
BICO BYGG OG INNREDNING AS []
1504 TILSYNSSEKSJONEN [('1504', 'DATE')]
PALETTE SOFTWARE AS []
PETTER LORANGE HANSEN []
HANSEN, NILS PETTER LORANGE [('HANSEN', 'ORG')]
STANDAL, JOAKIM STRANDBERG [('STANDAL', 'ORG')]
HOVDA, GEIR TREVERKET [('GEIR', 'ORG')]
OMNES, PÃÂL HELGE [('OMNES', 'GPE'), ('PÃ\x83Â', 'GPE'), ('L HELGE', 'GPE')]
SHM

In [11]:
nb = spacy.load("nb_dep_ud_sm")

for u in uppers:
    print(u,tag(nb,u))

REGION FINNMARK []
OSLO KOMMUNE BRANN- OG REDNINGSETATEN []
7080000243389 - MÃÂLLH. ÃÂKERN []
DAVID ANAND GOKSEM  951 31 231 []
LIFE IMPROVEMENT AS []
ANDERSEN, ROY DRAGE []
ALTECH AS []
HAUGEMYRA []
INTERIÃÂRRIKET ALTA AS []
KNUT EGIL OHNSTAD []
NATASJA BOURS VAN DEN BRINK []
HAMRE, PAUL []
STEINAR HESTHAG []
HJELTNES, GURI []
1503 AREALSEKSJONEN []
GJELDOKK, HÃÂVARD []
BICO BYGG OG INNREDNING AS []
1504 TILSYNSSEKSJONEN []
PALETTE SOFTWARE AS []
PETTER LORANGE HANSEN []
HANSEN, NILS PETTER LORANGE []
STANDAL, JOAKIM STRANDBERG []
HOVDA, GEIR TREVERKET []
OMNES, PÃÂL HELGE []
SHM ENABLING TECHNOLOGIES AS []
NORDEA BANK AB (PUBL), FILIAL I NORGE []
LANDKREDITT BANK AS []
HATLAND, MIKKEL KRISTOFFER []
KLINIKK FOR FOTTERAPI, AUTORISERT FOTTERAPEUT EVA []
AUTOLIFT AS []
VIKT CONSULTING AS []
MOLDEN, STIAN []
SYVERUD, NINA MARLEN []
JOHANSEN, KRISTINE IDA []
LAUGERUD, ARNE []
JOHANSEN, SINA MARLÃÂN []
AAKVIK, RUNAR []
SCHINDLER AS []
GRIMSÃÂEN []
MAY BRITT HOLTER []
GRIMSÃÂ

In [5]:
def mixed(df):
    for s in df:
        quoted = quoted_text(s)
        if (not quoted.isupper() and not quoted.islower()):
            print(quoted)

mixed(other)

Installert i
Sist brukt16.09.2018Installert iOption NV GTM378E
60 Abonnement
926 45 344FJELD, ROBERT133006/Fjeld, Robert04.06.2018AVD MARKETING-PU908 93 596KITTELSEN, GUNNAR1955...
77 63 23 94
Lokasjon
Created with Highcharts 4.2.7Kostnader siden mÃÂ¥nedsskifteFaste:263,-Faste:263,-Faste:263,-PÃÂ¥lÃÂ¸pt:63,...
77 65 07 27
GÃÂ¥ tilbake til forsiden
3 Abonnement
77 84 16 87
Sist brukt17.09.2018Installert iSamsung GALAXY A5 2016 SM-A510F
78 95 14 36
Page: 44651557
AdresseMARKVEIENÃÂ 42ÃÂ 9510 ÃÂ ALTA
ADMINISTRASJON598982370 abonnementFITJAR KOMMUNE ÃÂKONOMIAVDELING59898241 abonnementFITJAR KOMMUNE IK...
IKT AVDELING59898257 abonnementFITJAR KOMMUNE SKULEKONTORET59898262 abonnementFITJAR KOMMUNE RIMBARE...
ÃÂKONOMIAVDELING59898241 abonnementFITJAR KOMMUNE IKT AVDELING59898257 abonnementFITJAR KOMMUNE SKULE...
SÃÂLEVIK SKULE59898292 abonnementFITJAR KOMMUNE PLAN OG MILJÃÂ59898311 abonnementFITJAR KOMMUNE VAKTME...
RUSTENESTA59898435 abonnementFITJAR KOMMUNE FBB598984510 abonn

In [4]:
t.action.value_counts().tail(300)

click on "Avanserte rapporter"                                                                                           1
keypress <SPACE> on "search"                                                                                             1
click on "Abonnement ( 7 )"                                                                                              1
click on "(90016294)"                                                                                                    1
click on "HELGESEN TEKNISKE BYGG AS"                                                                                     1
click on "481 69 158"                                                                                                    1
click on "RÃÂINE, IDA CHRISTINE"                                                                                       1
click on "6 Avtaleprodukter"                                                                                             1
click on "406 22